In [5]:
#!/usr/bin/env python3
"""
Solve a CVRP instance (e.g. X-n101-k25) in VRPLIB format using PyVRP.

Usage
-----
python solve_cvrp_pyvrp.py path/to/X-n101-k25.vrp \
    --time_limit 30 \
    --seed 42
"""

import argparse
import time

from pyvrp import read, solve
from pyvrp.stop import MaxRuntime


def parse_args():
    parser = argparse.ArgumentParser(
        description="Solve a CVRP instance with PyVRP."
    )
    parser.add_argument(
        "instance",
        help="Path to the VRPLIB .vrp instance file (e.g. X-n101-k25.vrp)",
    )
    parser.add_argument(
        "--time_limit",
        type=float,
        default=30.0,
        help="Wall-clock time limit for the solver in seconds (default: 30.0).",
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=0,
        help="Random seed for PyVRP (default: 0).",
    )
    return parser.parse_args()


def compute_cumulative_distance(result, instance):
    """
    Example of a cumulative-style metric: sum over all routes of
    the prefix-sum of travel distances along each route.

    This is NOT PyVRP's built-in objective (total distance), just
    a custom metric for benchmarking.
    """
    solution = result.best

    # For standard CVRP with a single profile, the profile index is 0.
    dm = instance.distance_matrix(0)  # (num_locations x num_locations)

    cumulative_cost = 0

    for route in solution.routes():
        # schedule() gives you depot start, all clients, depot end, etc.
        visits = [visit.location for visit in route.schedule()]

        prefix_dist = 0
        for i in range(len(visits) - 1):
            frm = visits[i]
            to = visits[i + 1]
            prefix_dist += dm[frm, to]
            cumulative_cost += prefix_dist

    return cumulative_cost

def solve_instance(
        instance_path,
        time_limit=30.0,
        seed=0,
        round_func="round",
        display=False,
        verbose=True,
):
    """
    Programmatic wrapper to solve a CVRP instance with PyVRP.

    Parameters
    - instance_path: path to the .vrp file
    - time_limit: wall-clock time limit in seconds
    - seed: random seed
    - round_func: rounding function name passed to pyvrp.read (e.g. "round")
    - display: pass-through to pyvrp.solve
    - verbose: if True, print a short summary

    Returns a dict with keys:
    - result: the pyvrp result object
    - solve_time: wall-clock time used
    - total_distance: result.cost()
    - cumulative_distance: custom cumulative metric (compute_cumulative_distance)
    - instance: the loaded pyvrp instance
    """
    instance = read(instance_path, round_func=round_func)
    stop = MaxRuntime(time_limit)

    t0 = time.perf_counter()
    result = solve(instance, stop=stop, seed=seed, display=display)
    t1 = time.perf_counter()
    solve_time = t1 - t0

    cum_cost = compute_cumulative_distance(result, instance)

    if verbose:
        print("\n=== PyVRP result ===")
        print(result)
        print(f"\nWall-clock solve time: {solve_time:.3f} s")
        print(f"Best objective (total distance): {result.cost()}")
        print(f"Cumulative distance metric (custom): {cum_cost}")

    return {
        "result": result,
        "solve_time": solve_time,
        "total_distance": result.cost(),
        "cumulative_distance": cum_cost,
        "instance": instance,
    }

In [ ]:
sol = solve_instance(
    "Uchoa_et_al_2014/X-n101-k25.vrp",
    time_limit=30.0,
    seed=42,
    display=True,
    verbose=True,
)usage: pyvrp [-h] [--stats_dir STATS_DIR] [--sol_dir SOL_DIR] [--round_func {round,trunc,dimacs,exact,none}] [--config_loc CONFIG_LOC] --seed SEED [--num_procs NUM_PROCS] [--max_runtime MAX_RUNTIME]
             [--max_iterations MAX_ITERATIONS] [--no_improvement NO_IMPROVEMENT] [--per_client]
             instances [instances ...]

This program is a command line interface for solving VRPs, specified in VRPLIB format. The program can solve one or multiple such VRP instances, and outputs useful information in either case.

positional arguments:
  instances             One or more paths to the VRPLIB instance(s) to solve.

options:
  -h, --help            show this help message and exit
  --stats_dir STATS_DIR
                        Directory to store runtime statistics in, as CSV files (one per instance).
  --sol_dir SOL_DIR     Directory to store best observed solutions in, in VRPLIB format (one file per instance).
  --round_func {round,trunc,dimacs,exact,none}
                        Round function to apply for non-integral data. Default 'none'.
  --config_loc CONFIG_LOC
                        Optional parameter configuration file (in TOML format). These arguments replace the defaults if a file is passed; default parameters are used when this argument is not given.
  --seed SEED           Seed to use for reproducible results.
  --num_procs NUM_PROCS
                        Number of processors to use for solving instances. Default 1.

Stopping criteria:
  --max_runtime MAX_RUNTIME
                        Maximum runtime for each instance, in seconds.
  --max_iterations MAX_ITERATIONS
                        Maximum number of iterations for solving each instance.
  --no_improvement NO_IMPROVEMENT
                        Maximum number of iterations without improvement.
  --per_client
PS C:\Users\hmdis\Desktop\optimal-vs-heuristics-vrp> pyvrp --stats_dir
usage: pyvrp [-h] [--stats_dir STATS_DIR] [--sol_dir SOL_DIR] [--round_func {round,trunc,dimacs,exact,none}] [--config_loc CONFIG_LOC] --seed SEED [--num_procs NUM_PROCS] [--max_runtime MAX_RUNTIME]
             [--max_iterations MAX_ITERATIONS] [--no_improvement NO_IMPROVEMENT] [--per_client]
             instances [instances ...]
pyvrp: error: argument --stats_dir: expected one argument
PS C:\Users\hmdis\Desktop\optimal-vs-heuristics-vrp> pyvrp --stats_dir .
usage: pyvrp [-h] [--stats_dir STATS_DIR] [--sol_dir SOL_DIR] [--round_func {round,trunc,dimacs,exact,none}] [--config_loc CONFIG_LOC] --seed SEED [--num_procs NUM_PROCS] [--max_runtime MAX_RUNTIME]
             [--max_iterations MAX_ITERATIONS] [--no_improvement NO_IMPROVEMENT] [--per_client]
             instances [instances ...]
pyvrp: error: the following arguments are required: instances, --seed
PS C:\Users\hmdis\Desktop\optimal-vs-heuristics-vrp> 


PyVRP v0.12.1

Solving an instance with:
    1 depot
    100 clients
    100 vehicles (1 vehicle type)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best
H     859      5s |  47    27716    27595 |  36    27667    27482
H    1855     10s |  59    27689    27591 |  45    27569    27461
     2842     15s |  37    27732    27591 |  44    27646    27463
     3760     20s |  27    27826    27591 |  56    27657    27465
     4755     25s |  59    27711    27591 |  62    27580    27465

Search terminated in 30.00s after 5725 iterations.
Best-found solution has cost 27591.

Solution results
    # routes: 26
     # trips: 26
   # clients: 100
   objective: 27591
    distance: 27591
    duration: 27591
# iterations: 5725
    run-time: 30.00 seconds


=== PyVRP result ===
Solution results
    # routes: 26
     # trips: 26
   # clients: 100
   objective: 27591
    distance: 27591
    duration: 27591
# iterations: 5725
    

In [7]:
sol

{'result': Result(best=<pyvrp._pyvrp.Solution object at 0x000001B657AE5230>, stats=<pyvrp.Statistics.Statistics object at 0x000001B6561E7890>, num_iterations=5725, runtime=30.001957300119102),
 'solve_time': 30.019021699903533,
 'total_distance': 27591,
 'cumulative_distance': 86376,
 'instance': <pyvrp._pyvrp.ProblemData at 0x1b656173070>}